## Top attractions and what to see in Matera, Italy  

Introduction

Top Attractions in Matera


There are different website scraping libraries and packages in Python. One of the most common packages is BeautifulSoup and we will use it in this project. 
Package's main documentation page: http://beautiful-soup-4.readthedocs.io/en/latest/


Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [9]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
from sklearn.datasets.samples_generator import make_blobs
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
from bs4 import BeautifulSoup
import lxml
print('Libraries imported.')

Libraries imported.


<a id='item1'></a>

## Download, scrape and wrangle

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>to_explorer</em>, as shown below.

In [10]:
address = 'Matera, Italy'

geolocator = Nominatim(user_agent="mt_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Matera, Italy are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Matera, Italy are 40.668523, 16.601585.


#### Create a map of Matera with neighborhoods superimposed on top.

In [30]:
# create map of Matera using latitude and longitude values
map_matera = folium.Map(location=[latitude, longitude], zoom_start=12)
folium.Marker(location=[latitude, longitude],popup="Matera")
  
    
map_matera

Next, we are going to start utilizing the Foursquare API to explore the venues and segment them.

#### Define Foursquare Credentials and Version

In [12]:
CLIENT_ID = 'JBREGZ4UNA53HX43WMAD4TQ2X2XJWMX5DPHEZEIZHQA0ACNP' # your Foursquare ID
CLIENT_SECRET = 'VNS40KF3V4MGSWWAV0IGQINZIGIT1EQKNCWBFPOS3QF1JMOJ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JBREGZ4UNA53HX43WMAD4TQ2X2XJWMX5DPHEZEIZHQA0ACNP
CLIENT_SECRET:VNS40KF3V4MGSWWAV0IGQINZIGIT1EQKNCWBFPOS3QF1JMOJ


#### Let's explore trending venues in Matera.

First, let's create the GET request URL. Name your URL **url**.

In [13]:
LIMIT = 1000 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude,
    radius, 
    LIMIT)

# display URL
url


'https://api.foursquare.com/v2/venues/explore?&client_id=JBREGZ4UNA53HX43WMAD4TQ2X2XJWMX5DPHEZEIZHQA0ACNP&client_secret=VNS40KF3V4MGSWWAV0IGQINZIGIT1EQKNCWBFPOS3QF1JMOJ&v=20180605&ll=40.668523,16.601585&radius=1000&limit=1000'

Send the GET request and examine the resutls

In [14]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c7514511ed2196e47d915b7'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Matera',
  'headerFullLocation': 'Matera',
  'headerLocationGranularity': 'city',
  'totalResults': 47,
  'suggestedBounds': {'ne': {'lat': 40.67752300900001,
    'lng': 16.613428499191823},
   'sw': {'lat': 40.65952299099999, 'lng': 16.589741500808177}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c2dc202e307d13a894a0eda',
       'name': 'Piazza Vittorio Veneto',
       'location': {'address': 'Piazza Vittorio Veneto',
        'lat': 40.666604590423866,
        'lng': 16.606413844222335,
        'labeledLatLngs': [{'label': 'display',
          'lat': 40.6666045904

In [15]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [16]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Piazza Vittorio Veneto,Plaza,40.666605,16.606414
1,Bar Sottozero,Café,40.668856,16.606149
2,Locanda Di San Martino,Hotel,40.666679,16.608157
3,Cremeria dell angelo,Ice Cream Shop,40.667504,16.606040
4,Caffè Tripoli,Café,40.666854,16.606285


And how many venues were returned by Foursquare?

In [17]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

47 venues were returned by Foursquare.


<a id='item2'></a>

## Explore Venues in Matera

#### Let's check the size of the resulting dataframe

In [18]:
print(nearby_venues.shape)
nearby_venues

(47, 4)


,name,categories,lat,lng
0,Piazza Vittorio Veneto,Plaza,40.666605,16.606414
1,Bar Sottozero,Café,40.668856,16.606149
2,Locanda Di San Martino,Hotel,40.666679,16.608157
3,Cremeria dell angelo,Ice Cream Shop,40.667504,16.606040
4,Caffè Tripoli,Café,40.666854,16.606285
5,GROOVE,Gastropub,40.667495,16.605457
6,La Gatta Buia,Italian Restaurant,40.665834,16.609791
7,Ristorante Stano,Restaurant,40.670080,16.608670
8,Sextantio | Le Grotte della Civita,Hotel,40.667818,16.611844
9,Casa Cava,Music Venue,40.668489,16.609298


#### Let's find out how many unique categories can be curated from all the returned venues

In [19]:
print('There are {} uniques categories.'.format(len(nearby_venues['categories'].unique())))

There are 22 uniques categories.


<a id='item3'></a>

## Analyze Venues and Categories

#### Next, let's group rows by category and by taking the mean of the frequency of occurrence

In [20]:
# one hot encoding
matera_onehot = pd.get_dummies(nearby_venues[['name']], prefix="", prefix_sep="")

# add categories column back to dataframe
matera_onehot['categories'] = nearby_venues['categories'] 

#move categories column to the first position
categories = matera_onehot['categories']
matera_onehot.drop(labels=['categories'], axis=1,inplace = True)
matera_onehot.insert(0, 'categories', categories)
matera_onehot.head()


,categories,19a Buca Winery?,Antica Pasticceria Caffé Schiuma,Area8,Bar Sottozero,Birrificio 79,Caffè Lanfranchi,Caffè Ridola,Caffè Tripoli,Casa Cava,Castello Tramontano,Cinetix Cafè,Cremeria dell angelo,GROOVE,Hotel Nazionale,I Vizi degli Angeli,L'Arturo,La Cola Cola,La Focagna,La Gatta Buia,La Gelida Voglia,La Panca,Le Botteghe,Locanda Di San Martino,Malto & Luppolo,Musma,Oi Marì,Osteria Malatesta,Osteria Pico,Palazzo Lanfranchi,Palombaro Lungo,Pasthello,Piazza Duomo,Piazza San Francesco d'Assisi,Piazza Sedile,Piazza Vittorio Veneto,Piazzetta Pascoli,Pummarola,Ristorante Stano,Sassi di Matera,Sextantio | Le Grotte della Civita,Shibuya,Soul Kitchen,Terrazza Dell'Annunziata,Trattoria Lucana,Vicolo Cieco,Vittorio Veneto Caffé,panecotto
0,Plaza,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,Café,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Hotel,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Ice Cream Shop,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Café,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### Let's print each category along with the top venue

In [21]:
# let's group rows by categories and by taking the mean of the frequency of occurrence of each category

matera_grouped = matera_onehot.groupby('categories').mean().reset_index()
matera_grouped.head()


,categories,19a Buca Winery?,Antica Pasticceria Caffé Schiuma,Area8,Bar Sottozero,Birrificio 79,Caffè Lanfranchi,Caffè Ridola,Caffè Tripoli,Casa Cava,Castello Tramontano,Cinetix Cafè,Cremeria dell angelo,GROOVE,Hotel Nazionale,I Vizi degli Angeli,L'Arturo,La Cola Cola,La Focagna,La Gatta Buia,La Gelida Voglia,La Panca,Le Botteghe,Locanda Di San Martino,Malto & Luppolo,Musma,Oi Marì,Osteria Malatesta,Osteria Pico,Palazzo Lanfranchi,Palombaro Lungo,Pasthello,Piazza Duomo,Piazza San Francesco d'Assisi,Piazza Sedile,Piazza Vittorio Veneto,Piazzetta Pascoli,Pummarola,Ristorante Stano,Sassi di Matera,Sextantio | Le Grotte della Civita,Shibuya,Soul Kitchen,Terrazza Dell'Annunziata,Trattoria Lucana,Vicolo Cieco,Vittorio Veneto Caffé,panecotto
0,Art Gallery,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
1,Art Museum,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
2,Beer Store,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
3,Brewery,0.0,0.0,0.0,0.000000,1.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
4,Café,0.0,0.0,0.0,0.142857,0.0,0.142857,0.142857,0.142857,0.0,0.0,0.142857,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.0,0.0,0.142857,0.0


In [22]:
num_top_venues = 1

for cat in matera_grouped['categories']:
    print("----"+cat+"----")
    temp = matera_grouped[matera_grouped['categories']==cat].T.reset_index()
    temp.columns = ['name','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')
                          


----Art Gallery----
                 name  freq
0  Palazzo Lanfranchi   1.0


----Art Museum----
    name  freq
0  Musma   1.0


----Beer Store----
              name  freq
0  Malto & Luppolo   1.0


----Brewery----
            name  freq
0  Birrificio 79   1.0


----Café----
                    name  freq
0  Vittorio Veneto Caffé  0.14


----Cocktail Bar----
    name  freq
0  Area8   0.5


----Comfort Food Restaurant----
                name  freq
0  Osteria Malatesta   1.0


----Deli / Bodega----
        name  freq
0  panecotto   0.5


----Dessert Shop----
                               name  freq
0  Antica Pasticceria Caffé Schiuma   1.0


----Gastropub----
     name  freq
0  GROOVE   1.0


----Historic Site----
              name  freq
0  Palombaro Lungo  0.33


----Hotel----
              name  freq
0  Hotel Nazionale  0.33


----Ice Cream Shop----
        name  freq
0  Pasthello  0.25


----Italian Restaurant----
          name  freq
0  Le Botteghe  0.33


----Music Venue----
   

<a id='item4'></a>

## 4.Map

Finally, let's visualize the top choices

In [23]:
#Let us map the categories to colors

colors_array = cm.rainbow(np.linspace(0, 1, len(nearby_venues['categories'])))
rainbow = [colors.rgb2hex(i) for i in colors_array]
matera_grouped['RGB']='red', 'blue', 'yellow','green','black','white','orange','purple','#2686f5','#1996f3','#10a2f0','#04b0ed','#08bee9','#12c8e6','#1fd3e1','#2adddd','#37e6d8','#40ecd4','#4df3ce','#58f8c9','#62fbc4','#6efebe'
matera_grouped

,categories,19a Buca Winery?,Antica Pasticceria Caffé Schiuma,Area8,Bar Sottozero,Birrificio 79,Caffè Lanfranchi,Caffè Ridola,Caffè Tripoli,Casa Cava,Castello Tramontano,Cinetix Cafè,Cremeria dell angelo,GROOVE,Hotel Nazionale,I Vizi degli Angeli,L'Arturo,La Cola Cola,La Focagna,La Gatta Buia,La Gelida Voglia,La Panca,Le Botteghe,Locanda Di San Martino,Malto & Luppolo,Musma,Oi Marì,Osteria Malatesta,Osteria Pico,Palazzo Lanfranchi,Palombaro Lungo,Pasthello,Piazza Duomo,Piazza San Francesco d'Assisi,Piazza Sedile,Piazza Vittorio Veneto,Piazzetta Pascoli,Pummarola,Ristorante Stano,Sassi di Matera,Sextantio | Le Grotte della Civita,Shibuya,Soul Kitchen,Terrazza Dell'Annunziata,Trattoria Lucana,Vicolo Cieco,Vittorio Veneto Caffé,panecotto,RGB
0,Art Gallery,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.00,0.0,0.00,0.0,0.000000,0.00,0.00,0.000000,0.000000,0.0,0.0,0.00,0.0,0.000000,1.0,0.000000,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,red
1,Art Museum,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.00,0.0,0.00,0.0,0.000000,0.00,0.00,0.000000,0.000000,0.0,1.0,0.00,0.0,0.000000,0.0,0.000000,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,blue
2,Beer Store,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.00,0.0,0.00,0.0,0.000000,0.00,0.00,0.000000,0.000000,1.0,0.0,0.00,0.0,0.000000,0.0,0.000000,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,yellow
3,Brewery,0.0,0.0,0.0,0.000000,1.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.00,0.0,0.00,0.0,0.000000,0.00,0.00,0.000000,0.000000,0.0,0.0,0.00,0.0,0.000000,0.0,0.000000,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,green
4,Café,0.0,0.0,0.0,0.142857,0.0,0.142857,0.142857,0.142857,0.0,0.000000,0.142857,0.00,0.0,0.000000,0.00,0.0,0.00,0.0,0.000000,0.00,0.00,0.000000,0.000000,0.0,0.0,0.00,0.0,0.000000,0.0,0.000000,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.142857,0.000000,0.0,0.142857,0.0,black
5,Cocktail Bar,0.0,0.0,0.5,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.00,0.0,0.00,0.0,0.000000,0.00,0.00,0.000000,0.000000,0.0,0.0,0.00,0.0,0.000000,0.0,0.000000,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.5,0.000000,0.000000,0.000000,0.0,0.000000,0.0,white
6,Comfort Food Restaurant,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.00,0.0,0.00,0.0,0.000000,0.00,0.00,0.000000,0.000000,0.0,0.0,0.00,1.0,0.000000,0.0,0.000000,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,orange
7,Deli / Bodega,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.00,0.5,0.00,0.0,0.000000,0.00,0.00,0.000000,0.000000,0.0,0.0,0.00,0.0,0.000000,0.0,0.000000,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.5,purple
8,Dessert Shop,0.0,1.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.00,0.0,0.00,0.0,0.000000,0.00,0.00,0.000000,0.000000,0.0,0.0,0.00,0.0,0.000000,0.0,0.000000,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,#2686f5
9,Gastropub,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,1.0,0.000000,0.00,0.0,0.00,0.0,0.000000,0.00,0.00,0.000000,0.000000,0.0,0.0,0.00,0.0,0.000000,0.0,0.000000,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,#1996f3


In [31]:
# create map
map_visitmatera = folium.Map(location=[latitude, longitude], zoom_start=16)

for lat, lon, poi, cluster, col in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['name'], nearby_venues['categories'],matera_grouped['RGB']):
    label=folium.Popup(' Name: ' + str(poi) + ' Category: ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=col,
        fill=True,
        fill_color=col,
        fill_opacity=0.8).add_to(map_visitmatera)
        
map_visitmatera.add_child(folium.LayerControl())
map_visitmatera


<a id='item5'></a>